# Module P5: Python and SQL

**Purpose:** This project explores the use of SQL in Python scripts using sqlite3. The data used contains author information and book information.

**Notes:**

- Only one create script (02_create_tables) is used in the setup python script. All tables are dropped in the create sql script and data is inserted using the .to_sql method from pandas. SQL code in drop tables and insert records scripts are for reference only.
- sql_features scripts deletes rows from books table where Tolkien is the author and updates (corrects) row where year is incorrect.
- sql_queries contains scripts for aggregating, filtering, grouping, joining, and sorting data in tables. More info can be found in comments within each file. The output tables these scripts create will differ depending on whether the features script has been ran prior to executing.

In [31]:
import sqlite3
import pandas as pd
import pathlib

# Define the database file in the current root project directory
db_file = pathlib.Path("project.sqlite3")


def create_database():
    """Function to create a database. Connecting for the first time
    will create a new database file if it doesn't exist yet.
    Close the connection after creating the database
    to avoid locking the file."""
    try:
        conn = sqlite3.connect(db_file)
        conn.close()
        print("Database created successfully.")
    except sqlite3.Error as e:
        print("Error creating the database:", e)

def create_tables():
    """Function to read and execute SQL statements to drop existing table and create new ones.
    This will drop the books_sorted table if db03 was previously ran."""
    try:
        with sqlite3.connect(db_file) as conn:
            sql_file = pathlib.Path("sql_create", "02_create_tables.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            conn.executescript(sql_script)
            print("Tables created successfully.")
    except sqlite3.Error as e:
        print("Error creating tables:", e)


def insert_data_from_csv():
    """Function to use pandas to read data from CSV files (in 'data' folder)
    and insert the records into their respective tables."""
    try:
        author_data_path = pathlib.Path("data", "authors.csv")
        book_data_path = pathlib.Path("data", "books.csv")
        authors_df = pd.read_csv(author_data_path)
        books_df = pd.read_csv(book_data_path)
        with sqlite3.connect(db_file) as conn:
            # use the pandas DataFrame to_sql() method to insert data
            # pass in the table name and the connection
            authors_df.to_sql("authors", conn, if_exists="replace", index=False)
            books_df.to_sql("books", conn, if_exists="replace", index=False)
            print("Data inserted successfully.")
    except (sqlite3.Error, pd.errors.EmptyDataError, FileNotFoundError) as e:
        print("Error inserting data:", e)


def main():
    create_database()
    create_tables()
    insert_data_from_csv()


if __name__ == "__main__":
    main()


Database created successfully.
Tables created successfully.
Data inserted successfully.


**Here we can see the first 5 rows in the Authors table. To see whole table, view the project.sqlite3 file.**

In [32]:
pd.read_sql("SELECT * FROM authors", con=sqlite3.connect(db_file)).head()

,author_id,first,last
0,10f88232-1ae7-4d88-a6a2-dfcebb22a596,Harper,Lee
1,c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70,George,Orwell
2,e0b75863-866d-4db4-85c7-df9bb8ee6dab,F. Scott,Fitzgerald
3,7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d,Aldous,Huxley
4,8d8107b6-1f24-481c-8a21-7d72b13b59b5,J.D.,Salinger


In [33]:
import pathlib
import sqlite3

# Define the database file in the current root project directory
db_file = pathlib.Path("project.sqlite3")


def update_row():
    """Function to read and execute SQL statements to update row"""
    try:
        with sqlite3.connect(db_file) as conn:
            sql_file = pathlib.Path("sql_features", "update_records.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            conn.executescript(sql_script)
            print("Row updated successfully.")
    except sqlite3.Error as e:
        print("Error updating row:", e)


def remove_row():
    """Function to read and execute SQL statements to delete rows with Tolkien"""
    try:
        with sqlite3.connect(db_file) as conn:
            sql_file = pathlib.Path("sql_features", "delete_records.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            conn.executescript(sql_script)
            print("Row removed successfully.")
    except sqlite3.Error as e:
        print("Error removing row:", e)


def main():
    update_row()
    remove_row()


if __name__ == "__main__":
    main()

Row updated successfully.
Row removed successfully.


In [34]:
import pathlib
import sqlite3
import pandas as pd

# Define the database file in the current root project directory
db_file = pathlib.Path("project.sqlite3")


def aggregate():
    """Function to read and execute SQL statements to count total rows in books.
    This number will change based on whether or not db02_features.py has been run."""
    try:
        with sqlite3.connect(db_file) as conn:
            sql_file = pathlib.Path("sql_queries", "query_aggregation.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            x = conn.executescript(sql_script)
            # why do I have to execute the script twice like this for the cursor object to return count? No f***ing idea.
            x.execute(sql_script)
            print(f"Row aggregated successfully. Count: {x.fetchone()[0]}")
    except sqlite3.Error as e:
        print("Error aggregating row:", e)


def filter():
    """Function to read and execute SQL statements to filter rows in books that were only written by Tolkien.
    This number will change based on whether or not db02_features.py has been run."""
    try:
        with sqlite3.connect(db_file) as conn:
            sql_file = pathlib.Path("sql_queries", "query_filter.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            x = conn.executescript(sql_script)
            x.execute(sql_script)
            # list of tuples of each row after sql script executes
            booklist = x.fetchall()
            # list comprehension to get value of first index (title) in booklist
            books = [books[0] for books in booklist]
            print(f"Row filtered successfully.\n\tBooks written by Tolkien:{books}")
    except sqlite3.Error as e:
        print("Error filtering row:", e)


def sort():
    """Creates a new books table and sorts by year published"""
    try:
        with sqlite3.connect(db_file) as conn:
            # Selects all from books and sorts by year published #
            sql_file = pathlib.Path("sql_queries", "query_sorting.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            conn.executescript(sql_script)
            #######################################################
            # List of rows in books
            li = conn.executescript(sql_script).execute(sql_script).fetchall()
            # Convert list into dataframe
            df = pd.DataFrame(
                li, columns=["book_id", "title", "year_published", "author_id"]
            )
            # Convert dataframe to sql table
            df.to_sql("books_sorted", conn, if_exists="replace", index=False)
            ########################################################
            print("Table sorted successfully. See new table in db.")
    except sqlite3.Error as e:
        print("Error sorting table:", e)


def group():
    """Creates a new authors table and groups by first name"""
    try:
        with sqlite3.connect(db_file) as conn:
            # Selects all from books and sorts by year published #
            sql_file = pathlib.Path("sql_queries", "query_group_by.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            conn.executescript(sql_script)
            #######################################################
            # List of rows in books
            li = conn.executescript(sql_script).execute(sql_script).fetchall()
            # Convert list into dataframe
            df = pd.DataFrame(li, columns=["author_id", "first", "last"])
            # Convert dataframe to sql table
            df.to_sql("authors_grouped", conn, if_exists="replace", index=False)
            ########################################################
            print("Table grouped successfully. See new table in db.")
    except sqlite3.Error as e:
        print("Error grouping table:", e)


def join():
    """Creates a new table joining authors and books by author_id and sorts by yea published"""
    try:
        with sqlite3.connect(db_file) as conn:
            # Selects all from books and sorts by year published #
            sql_file = pathlib.Path("sql_queries", "query_join.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            conn.executescript(sql_script)
            #######################################################
            # List of rows in books
            li = conn.executescript(sql_script).execute(sql_script).fetchall()
            # Convert list into dataframe
            df = pd.DataFrame(
                li,
                columns=[
                    "book_id",
                    "title",
                    "year_published",
                    "books.author_id",
                    "authors.author_id",
                    "first",
                    "last",
                ],
            )
            # Convert dataframe to sql table
            df.to_sql("joined_tables", conn, if_exists="replace", index=False)
            ########################################################
            print("Tables joined successfully. See new table in db.")
    except sqlite3.Error as e:
        print("Error joining table:", e)


def main():
    aggregate()
    filter()
    sort()
    group()
    join()


if __name__ == "__main__":
    main()

Row aggregated successfully. Count: 8
Row filtered successfully.
	Books written by Tolkien:[]
Table sorted successfully. See new table in db.
Table grouped successfully. See new table in db.
Tables joined successfully. See new table in db.


#### Here are all tables

In [35]:
pd.read_sql("SELECT * FROM books", con=sqlite3.connect(db_file)).head(10)

,book_id,title,year_published,author_id
0,d6f83870-ff21-4a5d-90ab-26a49ab6ed12,To Kill a Mockingbird,1960,10f88232-1ae7-4d88-a6a2-dfcebb22a596
1,0f5f44f7-44d8-4f49-b8c4-c64d847587d3,1984,1949,c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70
2,f9d9e7de-c44d-4d1d-b3ab-59343bf32bc2,The Great Gatsby,1925,e0b75863-866d-4db4-85c7-df9bb8ee6dab
3,38e530f1-228f-4d6e-a587-2ed4d6c44e9c,Brave New World,1932,7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d
4,c2a62a4b-cf5c-4246-9bf7-b2601d542e6d,The Catcher in the Rye,1951,8d8107b6-1f24-481c-8a21-7d72b13b59b5
5,3a1d835c-1e15-4a48-8e8c-b12239604e98,Fahrenheit 451,1953,0cc3c8e4-e0c0-482f-b2f7-af87330de214
6,c6e67918-e509-4a6b-bc3a-979f6ad802f0,Pride and Prejudice,1813,4dca0632-2c53-490c-99d5-4f6d41e56c0e
7,ca8e64c3-1e67-47f5-82cc-3e4e30f63b75,Harry Potter and the Philosopher's Stone,1997,6b693b96-394a-4a1d-a4e2-792a47d7a568


In [36]:
pd.read_sql("SELECT * FROM authors", con=sqlite3.connect(db_file)).head(10)

,author_id,first,last
0,10f88232-1ae7-4d88-a6a2-dfcebb22a596,Harper,Lee
1,c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70,George,Orwell
2,e0b75863-866d-4db4-85c7-df9bb8ee6dab,F. Scott,Fitzgerald
3,7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d,Aldous,Huxley
4,8d8107b6-1f24-481c-8a21-7d72b13b59b5,J.D.,Salinger
5,0cc3c8e4-e0c0-482f-b2f7-af87330de214,Ray,Bradbury
6,4dca0632-2c53-490c-99d5-4f6d41e56c0e,Jane,Austen
7,16f3e0a1-24cb-4ed6-a50d-509f63e367f7,J.R.R.,Tolkien
8,06cf58ab-90f1-448d-8e54-055e4393e75c,J.R.R.,Tolkien
9,6b693b96-394a-4a1d-a4e2-792a47d7a568,J.K.,Rowling


In [37]:
pd.read_sql("SELECT * FROM books_sorted", con=sqlite3.connect(db_file)).head(10)

,book_id,title,year_published,author_id
0,c6e67918-e509-4a6b-bc3a-979f6ad802f0,Pride and Prejudice,1813,4dca0632-2c53-490c-99d5-4f6d41e56c0e
1,f9d9e7de-c44d-4d1d-b3ab-59343bf32bc2,The Great Gatsby,1925,e0b75863-866d-4db4-85c7-df9bb8ee6dab
2,38e530f1-228f-4d6e-a587-2ed4d6c44e9c,Brave New World,1932,7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d
3,0f5f44f7-44d8-4f49-b8c4-c64d847587d3,1984,1949,c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70
4,c2a62a4b-cf5c-4246-9bf7-b2601d542e6d,The Catcher in the Rye,1951,8d8107b6-1f24-481c-8a21-7d72b13b59b5
5,3a1d835c-1e15-4a48-8e8c-b12239604e98,Fahrenheit 451,1953,0cc3c8e4-e0c0-482f-b2f7-af87330de214
6,d6f83870-ff21-4a5d-90ab-26a49ab6ed12,To Kill a Mockingbird,1960,10f88232-1ae7-4d88-a6a2-dfcebb22a596
7,ca8e64c3-1e67-47f5-82cc-3e4e30f63b75,Harry Potter and the Philosopher's Stone,1997,6b693b96-394a-4a1d-a4e2-792a47d7a568


In [38]:
pd.read_sql("SELECT * FROM authors_grouped", con=sqlite3.connect(db_file)).head(10)

,author_id,first,last
0,7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d,Aldous,Huxley
1,e0b75863-866d-4db4-85c7-df9bb8ee6dab,F. Scott,Fitzgerald
2,c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70,George,Orwell
3,10f88232-1ae7-4d88-a6a2-dfcebb22a596,Harper,Lee
4,8d8107b6-1f24-481c-8a21-7d72b13b59b5,J.D.,Salinger
5,6b693b96-394a-4a1d-a4e2-792a47d7a568,J.K.,Rowling
6,16f3e0a1-24cb-4ed6-a50d-509f63e367f7,J.R.R.,Tolkien
7,4dca0632-2c53-490c-99d5-4f6d41e56c0e,Jane,Austen
8,0cc3c8e4-e0c0-482f-b2f7-af87330de214,Ray,Bradbury


In [39]:
pd.read_sql("SELECT * FROM joined_tables", con=sqlite3.connect(db_file)).head(10)

,book_id,title,year_published,books.author_id,authors.author_id,first,last
0,None,None,NaN,None,16f3e0a1-24cb-4ed6-a50d-509f63e367f7,J.R.R.,Tolkien
1,None,None,NaN,None,06cf58ab-90f1-448d-8e54-055e4393e75c,J.R.R.,Tolkien
2,c6e67918-e509-4a6b-bc3a-979f6ad802f0,Pride and Prejudice,1813.0,4dca0632-2c53-490c-99d5-4f6d41e56c0e,4dca0632-2c53-490c-99d5-4f6d41e56c0e,Jane,Austen
3,f9d9e7de-c44d-4d1d-b3ab-59343bf32bc2,The Great Gatsby,1925.0,e0b75863-866d-4db4-85c7-df9bb8ee6dab,e0b75863-866d-4db4-85c7-df9bb8ee6dab,F. Scott,Fitzgerald
4,38e530f1-228f-4d6e-a587-2ed4d6c44e9c,Brave New World,1932.0,7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d,7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d,Aldous,Huxley
5,0f5f44f7-44d8-4f49-b8c4-c64d847587d3,1984,1949.0,c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70,c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70,George,Orwell
6,c2a62a4b-cf5c-4246-9bf7-b2601d542e6d,The Catcher in the Rye,1951.0,8d8107b6-1f24-481c-8a21-7d72b13b59b5,8d8107b6-1f24-481c-8a21-7d72b13b59b5,J.D.,Salinger
7,3a1d835c-1e15-4a48-8e8c-b12239604e98,Fahrenheit 451,1953.0,0cc3c8e4-e0c0-482f-b2f7-af87330de214,0cc3c8e4-e0c0-482f-b2f7-af87330de214,Ray,Bradbury
8,d6f83870-ff21-4a5d-90ab-26a49ab6ed12,To Kill a Mockingbird,1960.0,10f88232-1ae7-4d88-a6a2-dfcebb22a596,10f88232-1ae7-4d88-a6a2-dfcebb22a596,Harper,Lee
9,ca8e64c3-1e67-47f5-82cc-3e4e30f63b75,Harry Potter and the Philosopher's Stone,1997.0,6b693b96-394a-4a1d-a4e2-792a47d7a568,6b693b96-394a-4a1d-a4e2-792a47d7a568,J.K.,Rowling
